<a href="https://colab.research.google.com/github/MithunSrinivas28/Fluxo_AI_Supplychain/blob/main/inference_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 1 — Mount Google Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# STEP 2 — Load Production Artifacts

In [8]:
import joblib

BASE_PATH = "/content/drive/MyDrive/fluxo_models/"

xgb_lower = joblib.load(BASE_PATH + "xgb_lower.pkl")
xgb_median = joblib.load(BASE_PATH + "xgb_median.pkl")
xgb_upper = joblib.load(BASE_PATH + "xgb_upper.pkl")
feature_columns = joblib.load(BASE_PATH + "feature_columns.pkl")

print("Models loaded successfully.")

Models loaded successfully.


# STEP 3 — Build Clean Prediction Function

In [9]:
import pandas as pd

def predict_demand(input_data: dict):

    # Convert input to DataFrame
    df = pd.DataFrame([input_data])

    # Apply one-hot encoding
    df = pd.get_dummies(
        df,
        columns=["zone", "warehouse", "category"],
        drop_first=True
    )

    # Align with training schema
    df = df.reindex(columns=feature_columns, fill_value=0)

    # Predict
    lower = xgb_lower.predict(df)[0]
    median = xgb_median.predict(df)[0]
    upper = xgb_upper.predict(df)[0]

    return {
        "forecast": float(median),
        "lower_bound": float(lower),
        "upper_bound": float(upper),
        "interval_width": float(upper - lower)
    }

In [10]:
def predict_demand(input_data: dict):

    df = pd.DataFrame([input_data])

    df = pd.get_dummies(
        df,
        columns=["zone", "warehouse", "category"],
        drop_first=True
    )

    df = df.reindex(columns=feature_columns, fill_value=0)

    lower = xgb_lower.predict(df)[0]
    median = xgb_median.predict(df)[0]
    upper = xgb_upper.predict(df)[0]

    # Enforce monotonicity
    lower = min(lower, median)
    upper = max(upper, median)

    return {
        "forecast": float(median),
        "lower_bound": float(lower),
        "upper_bound": float(upper),
        "interval_width": float(upper - lower)
    }

# STEP 4 — Test With Sample Input

In [11]:
sample_input = {
    "current_price": 92,
    "base_price": 100,
    "discount_percent": 8,
    "year_growth": 1.05,
    "month": 6,
    "is_festival": 0,
    "product_id": 12,
    "zone": "South",
    "warehouse": "B",
    "category": "dairy",
    "lag_1": 820,
    "lag_2": 790
}

result = predict_demand(sample_input)
result

{'forecast': 1048.8939208984375,
 'lower_bound': 844.4446411132812,
 'upper_bound': 1048.8939208984375,
 'interval_width': 204.44927978515625}